# SETTINGS 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#folder of dataset
import os
os.chdir('/content/drive/My Drive/')

In [ ]:
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.convolutional import AveragePooling2D
from keras.layers.core import Activation
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras.layers import Flatten
from keras.layers import Input
from keras.models import Model
from sklearn.model_selection import train_test_split
from keras.layers.core import Dense
from keras.models import Model
from keras.optimizers import Adam
from keras.layers import concatenate
import numpy as np
import argparse
import locale
import os
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from keras.applications import VGG16
import h5py
from random import *
from keras.preprocessing.image import ImageDataGenerator 
import random
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import ModelCheckpoint
from keras.layers.advanced_activations import LeakyReLU
from keras.layers import concatenate,Input,Dense,Conv2D,MaxPool2D,Dropout,Flatten
from keras.layers import Embedding
from keras.models import load_model,Model
from keras import backend as K
from keras.applications.vgg16 import VGG16
from keras.optimizers import SGD
from keras.models import load_model
import keras
import tensorflow as tf

# OPEN DATASET

In [ ]:
f=h5py.File('dataset_160.h5', 'r')
x_train = f['training']
x_valid = f['validation']

(123839, 160, 160, 3)


In [ ]:
f_label=h5py.File('label_160.h5', 'r')
y_train = f_label['label_training']
y_valid = f_label['label_validation']

(123839, 101)


In [ ]:
#Number of samples
n_val = x_valid.shape[0]
n_train = x_train.shape[0]

# CREATE BATCHES WITH RANDOM TRANSFORMATIONS

In [ ]:
#DATA AUGMENTATION
datagen = ImageDataGenerator(
   rotation_range=10,
   brightness_range=(0.7,1.5),
   horizontal_flip=True
)

apply_transfom=0

#BATCHES FOR TRAINING SET 
def generate_random_batches(X_train, y_data, size, stddev, batch_size, shape):
  start = True
  height=shape[0]
  width=shape[1]
  channel=shape[2]
  (n, h, w, c)=X_train.shape
  bs = batch_size
  indices = []
  iteration = 0
  index = randint(0, n)
  index_array = []
  apply_transfom=0

  if size % bs != 0:
    steps = size // bs + 1
    one_more_time = True 
  else:
    steps = size // bs
    one_more_time = False 
  print(X_train.shape)
  while True:
    
    #IN CASE OF THE LAST BATCH CONTAINS LESS THEN "batch_size" elements
    if iteration == steps-1 and one_more_time:
      num_elem = n - bs * (steps-1)
      X_batch = np.empty (shape=(num_elem, height, width, channel))
      y_batch = np.empty (shape=(num_elem, 101))
      X_batch_ordered = np.empty (shape=(num_elem, height, width, channel))
      y_batch_ordered = np.empty (shape=(num_elem, 101))

    else:
      num_elem = bs
      X_batch = np.empty (shape=(bs, height, width, channel))
      y_batch = np.empty (shape=(bs, 101))
      X_batch_ordered = np.empty (shape=(bs, height, width, channel))
      y_batch_ordered = np.empty (shape=(bs, 101))
    
    #BATCHES ARE TAKEN IN CIRCULAR ORDER STARTING FROM A RANDOM INDEX
    for i in range (num_elem):
      X_batch_ordered[i] = X_train[(index+i) % n]
      y_batch_ordered[i] = y_data[(index+i) % n]
    
    index_array = []
    for i in range (num_elem):
      index_array.insert(i, i)
    
    random.shuffle(index_array)
    
    #SHUFFLE ELEMENTS INSIDE THE BATCH AND APPLY TRANSFORMATIONS
    for i in range (num_elem):
      if random.randint(0, 4) == 1:
        X_batch[i] = datagen.random_transform(X_batch_ordered[index_array[i]])
      else:
        X_batch[i] = X_batch_ordered[index_array[i]]
      y_batch [i] = y_batch_ordered[index_array[i]]
  

    if iteration == steps-1:
      iteration = 0
      index = randint(0, n)
    else:
      iteration = iteration + 1
      index = (index + bs) % n


    yield X_batch, y_batch

# DEFINE MODEL

In [ ]:
import keras.backend as K
def contrastive_accuracy(y_true, y_pred):
    return  1.0-K.mean(K.max(abs(y_pred-y_true),axis=1)) 

In [ ]:
input_shape = (160, 160, 3)
model_mobilenet = keras.applications.mobilenet_v2.MobileNetV2(input_shape=input_shape , alpha = 0.75, include_top=False, weights='imagenet', pooling = 'avg')
model_mobilenet.trainable=True
input_dim=(160, 160, 3)
img_a = Input(shape=input_dim)

feat_vecs = model_mobilenet(img_a)
print(feat_vecs.shape)

d1=Dense(512,activation='relu')(feat_vecs)
d1=Dropout(rate=0.20)(d1)

d2=Dense(128,activation='relu')(d1)
d2=Dropout(rate=0.20)(d2)

d3=Dense(101,activation='softmax')(d2)

model=Model(inputs=img_a, outputs=d3)
print(model.summary())

optimizer = keras.optimizers.Adam()
model.compile(loss='categorical_crossentropy',optimizer = optimizer,metrics=['accuracy', contrastive_accuracy])

# TRAIN MODEL 

In [ ]:
# Early stopping
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor = 0.2, patience = 4, min_lr = 0.00001)

checkpoints_loss = ModelCheckpoint('mobilenet_age_Loss.h5', monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='min', period=1)
checkpoints_acc = ModelCheckpoint('mobilenet_age_Acc.h5', monitor='val_contrastive_accuracy', verbose=1, save_best_only=True, save_weights_only=False, mode='max', period=1)

callbacks = [EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True), reduce_lr, checkpoints_loss, checkpoints_acc]

In [ ]:
batch_size=100
print(n_val)
print(n_train)
resume=0

if n_train % batch_size != 0:
  steps_per_epoch = n_train//batch_size + 1 
else:
  steps_per_epoch = n_train//batch_size 


if n_val % batch_size != 0:
  validation_steps = n_val//batch_size + 1
else:
  validation_steps = n_val//batch_size

if resume:
  model.load_weights('gruppo8_mobilenet.h5')

model.fit(generate_random_batches(x_train, y_train, n_train, 2.0, batch_size, input_shape), 
              steps_per_epoch= steps_per_epoch,
              validation_data=(x_valid, y_valid),
              validation_steps=validation_steps,
              epochs=1000,
              callbacks=callbacks
              )